In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/air-passengers/AirPassengers.csv")
df.head()

In [ ]:
df["Month"] = pd.to_datetime(df["Month"])
df.head()


In [ ]:
df.set_index("Month", inplace=True)
df.columns = ["passengers"]
df.index.name = "date"
df.head()

In [ ]:
df.plot(figsize=(10,6))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
results = seasonal_decompose(df['passengers'])
results.plot();


In [ ]:
len(df)

In [ ]:
144-12

In [ ]:
train = df.iloc[:132]
test = df.iloc[132:]

In [ ]:

len(test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
scaler.fit(train)

In [ ]:


scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:


from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
scaled_train

In [ ]:
n_input = 2
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:
len(generator)

In [ ]:
scaled_train

In [ ]:
X,y = generator[0]

In [ ]:
n_input = 12
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:
scaled_train

In [ ]:
X,y = generator[0]

In [ ]:
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [ ]:


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
model.fit_generator(generator,epochs=50)

In [ ]:
loss = model.history.history['loss']
plt.plot(range(len(loss)),loss)

In [ ]:
first_eval_batch = scaled_train[-12:]

In [ ]:
first_eval_batch

In [ ]:
first_eval_batch = first_eval_batch.reshape((1, n_input, n_features))

In [ ]:
model.predict(first_eval_batch)

In [ ]:
scaled_test[0]

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

In [ ]:
current_batch.shape

In [ ]:


current_batch

In [ ]:
np.append(current_batch[:,1:,:],[[[99]]],axis=1)

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
scaled_test

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
true_predictions

In [ ]:
test

In [ ]:
test['Predictions'] = true_predictions

In [ ]:
test

In [ ]:
test.plot(figsize=(12,8))

In [ ]:
model.save('rnn model')

In [ ]:
from keras.models import load_model
new_model = load_model('rnn model')

In [ ]:
new_model.summary()